# ナップサック問題

ナップサック問題とは、容量制限のあるナップサックに価値と重量の異なるアイテムを詰め込み、価値の合計値を最大化する問題です。本チュートリアルでは、ナップサック問題をどのように解くかを説明します。

## ナップサック問題とは

ナップサック問題の具体例として、以下のストーリーを考えてみましょう。

> ある探検家が洞窟を冒険していると、思いがけず宝物を見つけました。

||Treasure A|Treasure B|Treasure C|Treasure D|Treasure E|Treasure F|
|-|-|-|-|-|-|-|
|Price|$5000|$7000|$2000|$1000|$4000|$3000|
|weight|800g|1000g|600g|400g|500g|300g|

> しかし、探検家が持っているのは容量2キロの小さなナップサックだけでした。当然、探検家はこのナップサックに出来るだけ価値の高い宝物を詰め込みたいと考えます。探検家は、どの宝物をナップサックに詰め込めば良いでしょうか？

このような状況下で最適解を求めるのがナップサック問題です。ナップサック問題はNP困難な整数計画問題として有名なものの1つです。

### 定式化

上の例を一般化して数理モデルを定式化してみましょう。ナップサックに入れるアイテム（宝物）の集合を $\{ 0, 1, \dots, i, \dots, N-1 \}$ とし、各アイテム $i$ の価値を $v_i$、重量を $w_i$ と書くことにします。

$$
v = \{v_0, v_1, \dots, v_i, \dots, v_{N-1}\}
$$

$$
w = \{w_0, w_1, \dots, w_i, \dots, w_{N-1}\}
$$

さらに、 アイテム $i$ を詰め込むかどうかを表現する決定変数として $x_i$ を導入します。 $x_i$ はアイテム $i$ をナップサックに詰め込む場合に1を取り、そうでない場合には0になるバイナリ変数です。また、ナップサックの容量を $W$ で表すことにします。

さて、ナップサックの容量制限を守りつつ、ナップサックに入れるアイテムの価値の合計を最大化するにはどうすればいいでしょうか？

まずは、ナップサックの容量制限を数式で表現してみましょう。これは、ナップサックに入れるアイテムの重量の合計値がナップサックの容量以下であることを数式で表現すれば良いです。すなわち、次のように表現することができます。

$$
\sum_{i=0}^{N-1} w_i x_i \leq W
$$

次は、ナップサックに入れるアイテムの価値の合計を数式で表現してみましょう。以下のように数式で表現することができます。

$$
\sum_{i=0}^{N-1} v_i x_i
$$

よって、ナップサック問題を数理モデルで表すと以下のようになります。

$$
\max \quad \sum_{i=0}^{N-1} v_i x_i \tag{1}
$$
$$
\mathrm{s.t.} \quad \sum_{i=0}^{N-1} w_i x_i \leq W \tag{2}
$$
$$
x_i \in \{0, 1\} \quad (\forall i \in \{0, 1, \dots, N-1\}) \tag{3}
$$

## JijModelingによる定式化

次に、JijModelingを使って上記の数理モデルを実装してみましょう。まずは、数理モデルに現れる変数およびパラメーターを定義します。

In [ ]:
import jijmodeling as jm

v = jm.Placeholder("v", ndim=1)
N = v.len_at(0, latex="N")
w = jm.Placeholder("w", ndim=1)
W = jm.Placeholder("W")
x = jm.BinaryVar("x", shape=(N,))
i = jm.Element("i", belong_to=(0, N))

ここで `v` はアイテム $i$ の価値 $v_i$ のリスト、 `w` はアイテム $i$ の重量 $w_i$ のリストを定義しており、 `N` はアイテムの個数、 `W` はナップサックの容量を定義しています。また、$x_i$ に相当するバイナリ変数として `x` を定義しています。 `i` は後の数理モデル構築のために用意された添字で、アイテム $i$ を相当します。

### 目的関数の実装

目的関数として式(1)を実装すると以下のようになります。

In [ ]:
problem = jm.Problem("Knapsack", sense=jm.ProblemSense.MAXIMIZE)
problem += jm.sum(i, v[i]*x[i])

ナップサック問題は目的関数を最大化する問題なので、 `sense=jm.ProblemSense.MAXIMIZE` を設定していることに注意してください。

### 制約条件の実装

制約条件として式(2)を実装すると以下のようになります。

In [ ]:
problem += jm.Constraint("weight", jm.sum(i, w[i]*x[i]) <= W)

これで数理モデルの実装は完了です。正しく数理モデルが実装されているかをLaTeX表示を通して確認してみましょう。

In [ ]:
problem

## インスタンスデータの準備

数理モデルが実装できたので、次はパラメーターに入力するインスタンスデータを準備します。ここでは `numpy` を使って100個のアイテムをランダム生成し、ナップサックの容量を100とするものとします。

In [ ]:
import numpy as np

inst_v = np.random.randint(5,30,100)
inst_w = inst_v + np.random.randint(-2,20,100)
inst_W = 100
instance_data = {"v": inst_v, "w": inst_w, "W": inst_W}

`inst_v` がアイテムの価値のリスト、 `inst_w` がアイテムの重量のリスト、 `inst_W` がナップサックの容量です。

## ナップサック問題を解く

さて、これでナップサック問題を解くためのすべての準備が完了しました。以下のコードを実行してナップサック問題を解いてみましょう。ここではJijZeptが提供しているソルバー `jijzept_solver` を利用してナップサック問題を解くこととします。

In [ ]:
import jijzept_solver

interpreter = jm.Interpreter(instance_data)
ommx_instance = interpreter.eval_problem(problem)

ommx_solution = jijzept_solver.solve(ommx_instance, time_limit_sec=4)
df = ommx_solution.decision_variables_df
df[df["value"] == 1.0]

## 解の可視化

最後に、上記で得られた解を用いて、詰め込んだアイテムの価値・重量を表示してみましょう。以下のコードでその結果を確認することができます。

In [ ]:
indices = df[df["value"] == 1]["subscripts"].to_list()

sum_w = 0
chosen_items_list = []
for j in indices:
    sum_w += inst_w[j[0]]
    chosen_items_list.append(j[0])

print("Values of chosen items: ", inst_v[chosen_items_list])
print("Weights of chosen items: ", inst_w[chosen_items_list])
print("Total value from objective: ", ommx_solution.objective)
print("Total weight: ", sum_w)

## 参考資料

- [Lucas, 2014, "Ising formulations of many NP problems"](https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full)